<a href="https://colab.research.google.com/github/chaoh818/appengine-php-wordpress-starter-project/blob/master/propCrawler_pelham_sdgnys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. parse roll call pdf and get list of props: Tax Map ID #
2. retrieve reponse from web query
3. parse and store in data structure
4. run simple DQ tests

* Based on view-source:https://townofpelham.sdgnys.com/report.aspx?file=vollocal/T000005/5544051630670001033000%20%20%20%200001.jpg&swiscode=554405&printkey=1630670001033000&sitetype=res&siteNum=1

[link text](https://)

# Prep Files

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
import os
from urllib.parse import urlparse
import requests
import threading
import shutil
import glob
from datetime import datetime
import pytz
from google.colab import drive
import time
from itertools import chain
import re

# Multithreaded File Downloader
class Downloader:
  def __init__(self, default_path, prefix = '', headers = {}, nthreads = 1):
    self.default_path = default_path
    self.prefix = prefix
    self.headers = headers
    self.nthreads = nthreads
    self.error_messages = []
    if not os.path.isdir(default_path):
      os.mkdir(self.default_path)

  def downloadInternal(self, source_path, target_path) -> bool:
    # print(f"begin to download {source_path}\n")
    response = requests.get(source_path, headers = self.headers, stream = True)
    if response.status_code != 200:
      # print(f"Unable to download {source_path}")
      self.error_messages.append(["Access error", source_path])
      return False
    with open(target_path, "wb") as f:
      for chunk in response.iter_content(chunk_size=1024):
        f.write(chunk)
    # print(f"Completed download for {source_path} in thread {threading.get_ident()}")
    return True

  def dedupList(self, source_list):
    dictionary = {}
    for item in source_list:
      if item in dictionary:
        self.error_messages.append(["Duplicate", item])
      else:
        dictionary[item] = 1
    return dictionary.keys()

  def generateTargetPath(self, source_path):
    clean_source_path = source_path.split('/')[-1]
    # use content before the first '.'
    clean_source_path = clean_source_path.split('.')[0]
    return self.default_path + self.prefix + clean_source_path

  def isValidPath(self, source_path) -> bool:
    # check if the source_path is a valide url
    result = urlparse(source_path)
    ret = all([result.scheme, result.netloc])
    if not ret:
      self.error_messages.append(["Invalid format", source_path])
      return False
    return True

  def download(self, source_file_list) -> bool:
    final_target_list = []
    final_source_list = []
    actual_download_list = []
    for f in self.dedupList(source_file_list):
      if not self.isValidPath(f):
        print(f"Unable to download path {f} because it's not valid")
        self.error_messages.append(["Invalid format", f])
        continue
      t = self.generateTargetPath(f)
      if os.path.exists(t):
        continue
      final_source_list.append(f)
      final_target_list.append(t)

    download_count = 0
    with ThreadPoolExecutor(max_workers = min(5, self.nthreads)) as executor:
      future_to_source = {executor.submit(
          self.downloadInternal, source_file, target_file): source_file \
          for source_file, target_file in \
                          zip(final_source_list, final_target_list)}
    for future in concurrent.futures.as_completed(future_to_source):
      source = future_to_source[future]
      try:
        res = future.result()
        if res:
          download_count +=1
          actual_download_list.append(source)
      except Exception as exc:
        print(f"{source} generated an exception: {exc}, exception type: \
        {type(exc).__name__}")
    if download_count == len(source_file_list):
      print(f"Downloaded all {download_count} files")
      return True
    else:
      print(f"Done trying to download {len(source_file_list)} files. \
      Actual downloaded {download_count} files\n")
      # print('\n'.join(actual_download_list))
      for e in self.error_messages:
        print(e[0], e[1])
      return False

def copy_files_recursively(src, dst):
    if not os.path.exists(dst):
        os.makedirs(dst)
    for item in os.listdir(src):
        src_item = os.path.join(src, item)
        dst_item = os.path.join(dst, item)
        if os.path.isdir(src_item):
            copy_files_recursively(src_item, dst_item)
        else:
            if not item.endswith('.gsheet'):
                shutil.copy2(src_item, dst_item)

def make_data_dir(dir_name):
  if not os.path.exists(dir_name):
    mode = 0o666
    os.mkdir(dir_name, mode)

def get_ny_local_time():
    ny_tz = pytz.timezone('America/New_York')
    ny_time = datetime.now(ny_tz)
    return ny_time

In [ ]:
# mount google drive
drive_mount_point = '/content/drive/MyDrive'
if os.path.ismount('/content/drive') and os.path.exists(drive_mount_point):
    print("Google Drive is already mounted.")
else:
    print("Google Drive is not mounted.")
    from google.colab import drive
    drive.mount('/content/drive')


Google Drive is not mounted.
Mounted at /content/drive


In [ ]:
# prepare data file and directories
copy_drive2colab = True
copy_colab2drive = False

# initiate or clean up downloads from colab at SOD
make_data_dir('/content/Props/')
make_data_dir('/content/Props/Downloads/')

# copy from drive to colab at SOD
if copy_drive2colab:
  copy_files_recursively('/content/drive/MyDrive/PropsSDG', '/content/Props/')

# copy from colab to drive at EOD
if copy_colab2drive:
  copy_files_recursively('/content/Props', '/content/drive/MyDrive/PropsSDG/')

# Get list of props

In [ ]:
# parse pdf file
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 93.2 MB/s eta 0:00:00


In [ ]:
# parse pdf file and find patterns
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

def find_patterns(text, pattern):
    matches = re.findall(pattern, text)
    return matches

pdf_path = "/content/Props/pelham.2024.roll.pdf"
pdf_text = extract_text_from_pdf(pdf_path)

# Regular expression pattern to find email addresses
pattern = r"\*+\s+[0-9.\-]+\s+\*+"
matches = find_patterns(pdf_text, pattern)

# extract the [0-9.\-]+ part in matches
tax_map_ids = [re.search(r"\*+\s+([0-9.\-]+)\s+\*+", match).group(1) for match in matches]

# print first 10
# for match in tax_map_ids[:10]:
#     print(match)


In [ ]:
# save to file
# with open('/content/Props/tax_map_ids.txt', 'w') as f:
#   for item in tax_map_ids:
#     f.write("%s\n" % item)

# read from file
with open('/content/Props/tax_map_ids.txt', 'r') as f:
  tax_map_ids = f.read().splitlines()


In [ ]:
# find index of element containing '164.37-1-2'
last_554403_idx = tax_map_ids.index('164.37-1-2')
# show 10 before and 10 after in the list around it
# for i in range(last_554403_idx-10, last_554403_idx+10):
#     print(tax_map_ids[i])

In [ ]:
# transform tax_map_id into printkey
# eg 158.68-1-10 -> 1580680001010000
def tmi2pk(tax_map_id):
  # tax_map_id = '158.68-1-10'
  printkey = ''
  # break tax_map_id to numbers separated by . and -
  for n in re.split(r'[.-]', tax_map_id):
    printkey += n.zfill(3)
  printkey = printkey[:6] + '0' + printkey[6:] + '000'
  return printkey

In [ ]:
filekeys = []
for i in range(len(tax_map_ids)):
  swiscode = '554403' if i <= last_554403_idx else '554405'
  printkey = tmi2pk(tax_map_ids[i])
  filekeys.append(f"{swiscode}{printkey}")

# Download all prop details

In [ ]:
# get response from url https://townofpelham.sdgnys.com/report.aspx?file=vollocal/T000005/5544051630670001033000%20%20%20%200001.jpg&swiscode=554405&printkey=1630670001033000&sitetype=res&siteNum=1


headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7,zh-TW;q=0.6",
    "Cache-Control": "max-age=0",
    "Connection": "keep-alive",
    "Cookie": "_ga=GA1.2.1416781400.1721524361; ASP.NET_SessionId=xkdqx355r0gi1tnd0u12hg55; TestCookie=test; LastSessID=xkdqx355r0gi1tnd0u12hg55; pubAccID=5544; _ga_5THW68DGLE=GS1.2.1722871029.2.0.1722871029.0.0.0; HasAgreedToDisclaimer=True",
    "Host": "townofpelham.sdgnys.com",
    "Referer": "https://townofpelham.sdgnys.com/disclaimer.aspx",
    "Sec-Ch-Ua": '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
    "Sec-Ch-Ua-Mobile": "?1",
    "Sec-Ch-Ua-Platform": '"Android"',
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Mobile Safari/537.36"
}

urls = []
for i in range(len(tax_map_ids)):
  swiscode = '554403' if i <= last_554403_idx else '554405'
  printkey = tmi2pk(tax_map_ids[i])
  url = f"https://townofpelham.sdgnys.com/report.aspx?file=vollocal/T000005/{swiscode}{printkey}.jpg&swiscode={swiscode}&printkey={printkey}&sitetype=res&siteNum=1"
  urls.append(url)




In [ ]:
downloader = Downloader("/content/Props/Downloads/", "report", headers, 4)
ret = downloader.download(urls)



Done trying to download 3494 files.       Actual downloaded 42 files



In [ ]:
!ls /content/Props/Downloads/ | wc

   3494    3494  101677


In [ ]:
!ls /content/drive/MyDrive/PropsSDG/Downloads/ | wc

   3494    3494  101677


# Parse parcel into df

```
pd.Series(df_counts).value_counts()
12    3030
11     347
9      117
```

10% missing sales table
3% missing all data, throw away

In [ ]:
# function sandbox
import pandas as pd
from bs4 import BeautifulSoup

def file_to_dfs(filekey):
  file_path = f'/content/Props/Downloads/report{filekey}'
  with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()
  soup = BeautifulSoup(content, 'lxml')
  tables = soup.find_all('table')
  # return tables

  dfs = []
  for i, table in enumerate(tables, start=1):
    dfs.append(table_to_df(table))
  return dfs

def table_to_df(table):
  # Extract headers
  headers = []
  for th in table.find_all('th'):
    headers.append(th.get_text().strip())
  # Extract rows
  rows = []
  for tr in table.find_all('tr'):
    cells = tr.find_all(['td', 'th'])
    row = [cell.get_text().strip().replace(' ', '').replace(':', '').replace('*', '') for cell in cells]
    if row:
      rows.append(row)
  df = pd.DataFrame(rows)
  return (df)

def process_4col_df(df0):
  df1 = pd.DataFrame(columns=['key', 'value'])
  df1['key'] = df0[0]
  df1['value'] = df0[1]
  df2 = pd.DataFrame(columns=['key', 'value'])
  df2['key'] = df0[2]
  df2['value'] = df0[3]
  df3 = pd.concat([df1, df2], ignore_index=True)
  df3 = df3[df3['key'] != '']
  df3 = df3.dropna()
  df3.reset_index(drop=True, inplace=True)
  return df3

def process_xpose_df(df0):
  #transpose df
  df1 = df0.transpose()
  df1 = df1.iloc[:, :2]
  # add column names 'key' and 'value'
  df1.columns = ['key', 'value']
  return df1

def process_tax_df(df0):
  df.columns = df0.iloc[0]
  df = df[1:]
  df = df.reset_index(drop=True)
  df['Amount'] = df['Amount'].str.replace('$', '').str.replace(',', '').astype(float)
  df = df.groupby('Year').sum()
  df.index = 'Tax' + df.index.astype(str)
  df = df.drop('Description', axis=1)
  df = df.reset_index()
  df.columns = ['key', 'value']
  return df

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

class ParcelDataParserSDG:
  def __init__(self, root_dir):
    self.root_dir = root_dir
    self.error_messages = []
    if not os.path.isdir(root_dir):
      self.error_messages.append(["File Missing", root_dir])
      for e in self.error_messages:
        print(e[0], e[1])

  def table_to_df(self, table):
    # Extract headers
    headers = []
    for th in table.find_all('th'):
      headers.append(th.get_text().strip())
    # Extract rows
    rows = []
    for tr in table.find_all('tr'):
      cells = tr.find_all(['td', 'th'])
      row = [cell.get_text().strip().replace(' ', '').replace(':', '').replace('*', '') for cell in cells]
      if row:
        rows.append(row)
    df = pd.DataFrame(rows)
    return (df)

  def process_4col_df(self, df0):
    df1 = pd.DataFrame(columns=['key', 'value'])
    df1['key'] = df0[0]
    df1['value'] = df0[1]
    df2 = pd.DataFrame(columns=['key', 'value'])
    df2['key'] = df0[2]
    df2['value'] = df0[3]
    df3 = pd.concat([df1, df2], ignore_index=True)
    df3 = df3[df3['key'] != '']
    df3 = df3.dropna()
    df3.reset_index(drop=True, inplace=True)
    return df3

  def process_xpose_df(self, df0):
    #transpose df
    df1 = df0.transpose()
    df1 = df1.iloc[:, :2]
    # add column names 'key' and 'value'
    df1.columns = ['key', 'value']
    return df1

  def process_tax_df(self, df0):
    df = df0[1:]
    df.columns = df0.iloc[0]
    df = df.reset_index(drop=True)
    df['Amount'] = df['Amount'].str.replace('$', '').str.replace(',', '').astype(float)
    df = df.groupby('Year').sum()
    df.index = 'Tax' + df.index.astype(str)
    df = df.drop('Description', axis=1)
    df = df.reset_index()
    df.columns = ['key', 'value']
    return df

  def tmi2pk(self, tax_map_id):
    printkey = ''
    for n in re.split(r'[.-]', tax_map_id):
      printkey += n.zfill(3)
    printkey = printkey[:6] + '0' + printkey[6:] + '000'
    return printkey

  def get_prop_class(self, swiscode, tax_map_id):
    res_df = None
    printkey = self.tmi2pk(tax_map_id)

    # property tables
    file_path = f'/content/Props/Downloads/report{swiscode}{printkey}'
    with open(file_path, 'r', encoding='utf-8') as file:
      content = file.read()
    soup = BeautifulSoup(content, 'lxml')
    tables = soup.find_all('table')
    df = self.process_4col_df(self.table_to_df(tables[1]))
    return df[df['key'] == 'PropertyClass']['value'].values[0]

  def parseParcel(self, swiscode, tax_map_id):
    res_df = None
    printkey = self.tmi2pk(tax_map_id)

    # property tables
    file_path = f'/content/Props/Downloads/report{swiscode}{printkey}'
    with open(file_path, 'r', encoding='utf-8') as file:
      content = file.read()
    soup = BeautifulSoup(content, 'lxml')
    tables = soup.find_all('table')

    df = self.process_4col_df(self.table_to_df(tables[1]))
    if df[df['key'] == 'PropertyClass']['value'].values[0] not in ['210-1FamilyRes','220-2FamilyRes']:
      return None

    if len(tables) <= 9:
      return None
    if len(tables) == 11:
      tables = tables[1:]
    if len(tables) == 12:
      tables = tables[1:4] + tables[5:]

    # 4-col: 0 1 2 3; 2-col-xpose: 5; 3-col-xpose: 9
    df = self.process_tax_df(self.table_to_df(tables[9]))
    res_df = df
    # enumerate first 4 elements in tables
    for i, table in enumerate(tables[:4]):
      df = self.process_4col_df(self.table_to_df(table))
      res_df = pd.concat([res_df, df], ignore_index=True)

    df = self.process_xpose_df(self.table_to_df(tables[5]))
    res_df = pd.concat([res_df, df], ignore_index=True)

    # prepend id
    id_data = {
        'key': ['ID'],
        'value': [f'{swiscode}{printkey}']
    }
    id_df = pd.DataFrame(id_data)
    res_df = pd.concat([id_df, res_df], ignore_index=True)

    res_df = res_df.transpose()
    res_df.reset_index(drop=True, inplace=True)
    res_df.columns = res_df.iloc[0]
    res_df = res_df[1:]
    return res_df

In [ ]:
with open('/content/Props/tax_map_ids.txt', 'r') as f:
  tax_map_ids = f.read().splitlines()
last_554403_idx = tax_map_ids.index('164.37-1-2')
parser = ParcelDataParserSDG("/content/Props/Downloads/")
master_df = None
# add error handling
try:
  for i, tax_map_id in enumerate(tax_map_ids):
    swiscode = '554403' if i <= last_554403_idx else '554405'
    df = parser.parseParcel(swiscode, tax_map_id)
    if master_df is None:
      master_df = df
    elif df is not None:
      master_df = pd.concat([master_df, df], ignore_index=True, sort=False)
except Exception as e:
  print(f"Error occurred at {i}: {e}")
master_df

,ID,Tax2023,Tax2024,RollSection,Swis,TaxMapID#,PropertyClass,Site,InAg.District,SitePropertyClass,...,FuelType,WaterSupply,HeatType,CentralAir,Type,Size,Tax2012,Tax2013,Tax2021,Tax2022
0,5544031580680001001000,21115.46,8687.95,Taxable,554403,158.68-1-1,210-1FamilyRes,RES1,No,210-1FamilyRes,...,Oil,Comm/public,Hotair,No,Primary,50 × 110,NaN,NaN,NaN,NaN
1,5544031580680001002000,13727.86,5641.81,Taxable,554403,158.68-1-2,210-1FamilyRes,RES1,No,210-1FamilyRes,...,NaturalGas,Comm/public,Hotwtr/stm,No,Primary,100 × 108,NaN,NaN,NaN,NaN
2,5544031580680001003000,23154.4,9642.58,Taxable,554403,158.68-1-3,210-1FamilyRes,RES1,No,210-1FamilyRes,...,Oil,Comm/public,Hotwtr/stm,Yes,Primary,50 × 104,NaN,NaN,NaN,NaN
3,5544031580680001004000,17455.7,7776.27,Taxable,554403,158.68-1-4,210-1FamilyRes,RES1,No,210-1FamilyRes,...,NaturalGas,Comm/public,Hotwtr/stm,No,Primary,50 × 99,NaN,NaN,NaN,NaN
4,5544031580680001005000,17790.87,7915.7,Taxable,554403,158.68-1-5,210-1FamilyRes,RES1,No,210-1FamilyRes,...,Oil,Comm/public,Hotwtr/stm,No,Primary,50 × 96,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2990,5544051640610001003000,NaN,NaN,WhollyExem,554405,164.61-1-3,210-1FamilyRes,RES1,No,210-1FamilyRes,...,Oil,Comm/public,Hotwtr/stm,No,Primary,150 × 123,2022.67,2008.56,NaN,NaN
2991,5544051660270002064000,1084.73,1099.5,WhollyExem,554405,166.27-2-64,210-1FamilyRes,RES1,No,210-1FamilyRes,...,NaturalGas,Comm/public,Hotair,Yes,Primary,75 × 166,NaN,NaN,NaN,NaN
2992,5544051670290001041000,737.97,747.7,WhollyExem,554405,167.29-1-41,210-1FamilyRes,RES1,No,210-1FamilyRes,...,Oil,Comm/public,Hotwtr/stm,No,Primary,120 × 70,NaN,NaN,NaN,NaN
2993,5544051670470001006000,635.85,638.19,WhollyExem,554405,167.47-1-6,210-1FamilyRes,RES1,No,210-1FamilyRes,...,NaturalGas,Comm/public,Hotair,Yes,Primary,148 × 130,NaN,NaN,NaN,NaN


# Data analysis and cleanup


In [ ]:
import pandas as pd
import numpy as np
# store master_df in a file
# master_df.to_csv('/content/drive/MyDrive/PropsSDG/master_df.csv')
# master_df.to_csv('/content/Props/master_df.csv')
master_df = pd.read_csv('/content/Props/master_df.csv', index_col=0)
# master_df.columns = master_df.columns.str.replace('-', '')
# master_df.columns = master_df.columns.str.replace('/', '')
# master_df.columns = master_df.columns.str.replace('(', '')
# master_df.columns = master_df.columns.str.replace(')', '')

In [ ]:
# keep only these columns in master_df
cols = ['ID','Tax2023','Tax2024','Swis','PropertyClass','ZoningCode','NeighborhoodCode','TotalAcreageSize','LandAssessment','TotalAssessment','LivingArea','SecondStoryArea','AdditionalStoryArea','FinishedBasement','FinishedRecRoom','FirstStoryArea','HalfStoryArea','34StoryArea','NumberofStories','FinishedAreaOverGarage','BuildingStyle','Bedrooms','Fireplaces','PorchType','BasementGarageCap','OverallCondition','YearBuilt','BathroomsFullHalf','Kitchens','BasementType','PorchArea','AttachedGarageCap','OverallGrade','SewerType','Utilities','FuelType','WaterSupply','HeatType','CentralAir','Type']
master_df = master_df[cols]

In [ ]:
# for generating list of columns, replace in string , with ','
astr = 'LandAssessment,TotalAssessment'
astr.replace(',', '\',\'')


"LandAssessment','TotalAssessment"

In [ ]:
def extract_dollar_values(val, idx):
  pattern = r'\$\d{1,3}(?:,\d{3})*'
  matches = re.findall(pattern, val)
  matches = [float(match.replace('$', '').replace(',', '')) for match in matches]
  if len(matches) < idx + 1:
    return np.nan
  return matches[idx]

def extract_integer_values(val, idx):
  pattern = r'\d{1,3}(?:,\d{3})*'
  matches = re.findall(pattern, val)
  matches = [float(match.replace('$', '').replace(',', '')) for match in matches]
  if len(matches) < idx + 1:
    return -1
  return matches[idx]

def extract_decimal_values(val, idx):
  pattern = r'\d+\.\d+'
  matches = re.findall(pattern, val)
  if len(matches) == 0:
    pattern = r'\d+'
    matches = re.findall(pattern, val)
  matches = [float(match.replace('$', '').replace(',', '')) for match in matches]
  if len(matches) < idx + 1:
    return -1
  return matches[idx]

def extract_acre_values(val):
  pattern = r'\d+\.\d+'
  matches = re.findall(pattern, val)
  if len(matches) == 0:
    pattern = r'\d+'
    matches = re.findall(pattern, val)
  matches = [float(match.replace('$', '').replace(',', '')) for match in matches]
  if len(matches) == 2:
    return matches[0] * matches[1] / 43560.0
  return matches[0]


In [ ]:
# for these columns, match pattern 2024-Tentative$234,0002023-$156,000
dollar_cols = ['LandAssessment','TotalAssessment']
sqft_cols = ['LivingArea','SecondStoryArea','AdditionalStoryArea','FinishedBasement','FinishedRecRoom','FirstStoryArea','HalfStoryArea','34StoryArea','FinishedAreaOverGarage','AttachedGarageCap']

for col in dollar_cols:
  master_df[f'{col}23'] = master_df[col].apply(extract_dollar_values, idx=1)
  master_df[f'{col}24'] = master_df[col].apply(extract_dollar_values, idx=0)
  master_df = master_df.drop(col, axis=1)

for col in sqft_cols:
  master_df[col] = master_df[col].apply(extract_integer_values, idx=0)

master_df['TotalAcreage/Size'] = master_df['TotalAcreage/Size'].apply(extract_acre_values)

master_df['PorchArea'] = master_df['PorchArea'].str.replace(',', '').astype('float64')
master_df['BathroomsFull'] = master_df['BathroomsFullHalf'].apply(extract_integer_values, idx=0)
master_df['BathroomsHalf'] = master_df['BathroomsFullHalf'].apply(extract_integer_values, idx=1)
master_df.drop(columns=['BathroomsFullHalf'], inplace=True)


<ipython-input-10-b99113cbc87d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  master_df[f'{col}23'] = master_df[col].apply(extract_dollar_values, idx=1)
<ipython-input-10-b99113cbc87d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  master_df[f'{col}24'] = master_df[col].apply(extract_dollar_values, idx=0)


KeyError: 'TotalAcreage/Size'

In [ ]:
master_df

,ID,Tax2023,Tax2024,Swis,PropertyClass,ZoningCode,NeighborhoodCode,TotalAcreageSize,LivingArea,SecondStoryArea,...,Utilities,FuelType,WaterSupply,HeatType,CentralAir,Type,LandAssessment23,LandAssessment24,TotalAssessment23,TotalAssessment24
0,5544031580680001001000,21115.46,8687.95,554403,210-1FamilyRes,RA3,00001-ChesterPark,50x110,1993.0,870.0,...,Electric,Oil,Comm/public,Hotair,No,Primary,159000.0,238000.0,810000.0,850000.0
1,5544031580680001002000,13727.86,5641.81,554403,210-1FamilyRes,RA3,00001-ChesterPark,100x108,1058.0,0.0,...,Gas&elec,NaturalGas,Comm/public,Hotwtr/stm,No,Primary,218500.0,328000.0,526000.0,552000.0
2,5544031580680001003000,23154.4,9642.58,554403,210-1FamilyRes,RA3,00001-ChesterPark,50x104,2001.0,897.0,...,Gas&elec,Oil,Comm/public,Hotwtr/stm,Yes,Primary,156000.0,234000.0,899000.0,944000.0
3,5544031580680001004000,17455.7,7776.27,554403,210-1FamilyRes,RA3,00001-ChesterPark,50x99,1456.0,728.0,...,Gas&elec,NaturalGas,Comm/public,Hotwtr/stm,No,Primary,154000.0,231000.0,725000.0,761000.0
4,5544031580680001005000,17790.87,7915.7,554403,210-1FamilyRes,RA3,00001-ChesterPark,50x102,1519.0,0.0,...,Gas&elec,Oil,Comm/public,Hotwtr/stm,No,Primary,156000.0,234000.0,738000.0,775000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2990,5544051640610001003000,NaN,NaN,554405,210-1FamilyRes,RAA,00009,150x123,9572.0,4552.0,...,Gas&elec,Oil,Comm/public,Hotwtr/stm,No,Primary,359000.0,538000.0,3031000.0,3183000.0
2991,5544051660270002064000,1084.73,1099.5,554405,210-1FamilyRes,RA,00011,75x166,2774.0,0.0,...,Gas&elec,NaturalGas,Comm/public,Hotair,Yes,Primary,254000.0,381000.0,1213000.0,1274000.0
2992,5544051670290001041000,737.97,747.7,554405,210-1FamilyRes,RA,00013,120x70,2080.0,1300.0,...,Gas&elec,Oil,Comm/public,Hotwtr/stm,No,Primary,240500.0,361000.0,1229000.0,1290000.0
2993,5544051670470001006000,635.85,638.19,554405,210-1FamilyRes,RAAA,00017,148x130,3217.0,1884.0,...,Gas&elec,NaturalGas,Comm/public,Hotair,Yes,Primary,366500.0,550000.0,1049000.0,1101000.0


In [ ]:
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
import os
from urllib.parse import urlparse
import requests
import threading
import shutil
import glob
from datetime import datetime
import pytz
from google.colab import drive
import time
from itertools import chain

# mount google drive
drive_mount_point = '/content/drive/MyDrive'
if os.path.ismount('/content/drive') and os.path.exists(drive_mount_point):
    print("Google Drive is already mounted.")
else:
    print("Google Drive is not mounted.")
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
# shutil.copy2('/content/Props/master_df.csv', '/content/drive/MyDrive/Props/master_df.csv')
shutil.copy2('/content/drive/MyDrive/Props/master_df.csv', '/content/Props/master_df.csv')

In [ ]:
master_df = pd.read_csv('/content/Props/master_df.csv', index_col=0)
# print number of rows
print(master_df.shape)

In [ ]:
# find columns that are 90% or more NULL or NaN
null_cols = master_df.columns[master_df.isnull().mean() >= 0.9]
# drop these columns
master_df = master_df.drop(null_cols, axis=1)
print(master_df.shape)

In [ ]:
unique_cols = master_df.columns[master_df.nunique() == 1]
unique_cols


Index([], dtype='object', name=0)